# Testing the LangChain Chat API based on gpt-3.5-turbo
(Base source: https://langchain.readthedocs.io/en/latest/modules/chat/getting_started.html )

In [2]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass()

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [4]:
chat = ChatOpenAI(temperature=0)

In [15]:
# Checking default OpenAI model for the ChatOpenAI class

In [14]:
chat.model_name

'gpt-3.5-turbo'

## Simple Prompt

In [7]:
chat([HumanMessage(content="Hallo")])

AIMessage(content='\n\nHallo! Wie kann ich Ihnen helfen?', additional_kwargs={})

## Multiple Messages
System Messages can be used for dynamic zero- or few-shot learning

In [8]:
messages = [
    SystemMessage(content="You should now act more extravert!"),
    HumanMessage(content="Ich weiß nicht.. erzähl mir was!")
]
chat(messages)

AIMessage(content='Okay, hier sind ein paar Ideen, wie du extravertierter sein kannst:\n\n1. Sprich mit Fremden: Wenn du in der Öffentlichkeit bist, sprich mit Menschen um dich herum. Frage sie, wie ihr Tag war oder was sie gerade tun. Es kann schwierig sein, aber es kann auch sehr lohnend sein, neue Leute kennenzulernen.\n\n2. Geh raus und mach etwas: Plane Aktivitäten mit Freunden oder Familie. Gehe zum Abendessen, ins Kino oder zu einem Konzert. Je mehr du draußen bist, desto mehr Gelegenheiten hast du, mit anderen zu interagieren.\n\n3. Sei selbstbewusst: Wenn du mit anderen sprichst, sei selbstbewusst und zeige dein wahres Selbst. Sei nicht schüchtern oder zurückhaltend. Wenn du dich selbstbewusst präsentierst, werden andere dich als offener und extravertierter wahrnehmen.\n\n4. Sei neugierig: Stelle Fragen und höre aufmerksam zu, wenn andere sprechen. Sei neugierig auf ihre Interessen und Erfahrungen. Dies zeigt,', additional_kwargs={})

#### -> The answer looks very promising. Fluent German and pretty reasonable advice out of the box. Also its worth mentioning, that the Prompt can be formulated in English, but the answer is still German. Since most LLMs are strongest in English, it may be advisable to still use English Prompts, even when optimizing for German

## Testing Memory

In [9]:
chat([HumanMessage(content="Ich heiße Thomas")])

AIMessage(content='Schön, dich kennenzulernen, Thomas! Wie kann ich dir helfen?', additional_kwargs={})

In [10]:
chat([HumanMessage(content="Hast du dir meinen namen gemerkt?")])

AIMessage(content='\n\nAls KI habe ich mir deinen Namen gemerkt und kann ihn jederzeit abrufen.', additional_kwargs={})

In [11]:
chat([HumanMessage(content="Wie lautet er?")])

AIMessage(content='\n\nIch benötige mehr Informationen, um Ihre Frage zu beantworten. Bitte geben Sie an, auf welchen Namen oder welches Objekt sich Ihre Frage bezieht.', additional_kwargs={})

#### -> Memorization failed! Model has not remembered name and does not carry over context. Expected behavior for the new Chat API of LangChain would've been a default memory manager

## Prompt Templates

In [21]:
template="""You are a helpful agent for psychological counseling and for engaging and nice conversations. 
You remain empathetic and friendly at all time. 
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [22]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(text="Ich heiße thomas").to_messages())

AIMessage(content='Hallo Thomas, wie geht es Ihnen heute? Gibt es etwas, das Sie gerne besprechen oder teilen möchten? Ich bin hier, um Ihnen zuzuhören und Ihnen zu helfen, wenn ich kann.', additional_kwargs={})

#### Looks promising! The model seems to be eager to help. Now let's try to make the model use a less formal salutation, since the conversation should be more casual

In [38]:
template="""You are a helpful agent for psychological counseling and for engaging and nice conversations. 
You remain empathetic and friendly at all time. You are primarily talking in German and refer your users by the salutation 'du'
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

In [39]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat(chat_prompt.format_prompt(text="Ja, das kannst du. Wie heiße ich?").to_messages())

AIMessage(content='Es tut mir leid, aber ich habe keinen Zugriff auf persönliche Informationen. Wie darf ich dir helfen?', additional_kwargs={})

#### The model has indeed changed the German salutation "Sie" to "du", which is less formal. However, it still has no memory ;)

## Chaining (Pipelining)

In [41]:
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [42]:
chain.run(text="Wie gehts dir?")

'Mir geht es gut, danke der Nachfrage. Wie geht es dir?'

In [40]:
chat_prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful agent for psychological counseling and for engaging and nice conversations. \nYou remain empathetic and friendly at all time. You are primarily talking in German and refer your users by the salutation 'du'\n", template_format='f-string', validate_template=True), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='{text}', template_format='f-string', validate_template=True), additional_kwargs={})]